<a href="https://colab.research.google.com/github/NaughtyFishies/mines_PE_alum/blob/main/10_ingest_list_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

10_ingest_list_docs.ipynb
Mines Alumni Project

Goal:
- Enumerate Dropbox folder via API
- Compute doc_id as hash. Store file metadata.
- Done when: /raw mirrors manifest with doc hashes logged

In [44]:
import hashlib
import os
import json
from datetime import datetime
import dropbox

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Run 00_boostrap-env to install dependencies and ensure that file structure exists:

In [46]:
%run /content/drive/MyDrive/mines-alum/00_bootstrap_env.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory structure ready:
raw        -> /content/drive/MyDrive/mines-alum/raw
interim    -> /content/drive/MyDrive/mines-alum/interim
processed  -> /content/drive/MyDrive/mines-alum/processed
profiles   -> /content/drive/MyDrive/mines-alum/profiles
index      -> /content/drive/MyDrive/mines-alum/index
logs       -> /content/drive/MyDrive/mines-alum/logs
config     -> /content/drive/MyDrive/mines-alum/config
[2025-10-24 01:30:06] Wrote sample log to Drive
[2025-10-24 01:30:06] Wrote environment report to Drive


In [47]:
dbx = init_dbx()

In [48]:
def compute_doc_id(path):
    return hashlib.sha256(path.encode("utf-8")).hexdigest()[:16]

def enumerate_dropbox_folder(dbx, folder):
    entries = []
    has_more = True
    cursor = None
    result = dbx.files_list_folder(folder, recursive=True)
    entries.extend(result.entries)
    while result.has_more:
        result = dbx.files_list_folder_continue(result.cursor)
        entries.extend(result.entries)

    manifest = []
    for entry in entries:
        if isinstance(entry, dropbox.files.FileMetadata):
            doc_id = compute_doc_id(entry.path_lower)
            metadata = {
                "doc_id": doc_id,
                "dropbox_path": entry.path_lower,
                "name": entry.name,
                "size": entry.size,
                "client_modified": entry.client_modified.isoformat(),
                "server_modified": entry.server_modified.isoformat(),
                "rev": entry.rev,
            }
            manifest.append(metadata)
    return manifest

Create manifest from dropbox API:

In [49]:
DROPBOX_FOLDER = "/Hall of Fame Committee"
manifest_path = f"{DIRS['index']}/manifest.json"

manifest = enumerate_dropbox_folder(dbx, DROPBOX_FOLDER)
print(f"Found {len(manifest)} files in Dropbox.")

Found 380 files in Dropbox.


In [50]:
with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2)

print("Manifest saved to:", manifest_path)

for item in manifest:
    local_path = os.path.join(DIRS["raw"], item["name"])
    open(local_path, "a").close()

Manifest saved to: /content/drive/MyDrive/mines-alum/index/manifest.json
